In [ ]:
#@title Colab setup
import os, sys, subprocess
if "google.colab" in sys.modules:
  cmd = "pip install --upgrade biocircuits bokeh-catplot watermark blackcellmagic intersect"
  process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()
# ------

import scipy.integrate
import bokeh.io
import numpy as np
from intersect import intersection

import biocircuits
import numba

from skimage import measure
from intersect import intersection
import sympy as sp
import pylab as pl

import bokeh.io
import bokeh.plotting
import bokeh.palettes
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.models import Range1d
from bokeh.io import export_svgs

bokeh.io.output_notebook()

In [ ]:
# Some auxiliary functions

def JoinMatrices(A,B):

  # Function to create two matrices of differense size by placing them in a diagonal [A, 0 ; 0, B]
  n1, m1 = A.shape
  n2, m2 = B.shape
  ZA = np.zeros((n1,m2))
  ZB = np.zeros((n2,m1))
  tmp1 = np.hstack((A,ZA))
  tmp2 = np.hstack((ZB,B))
  return np.vstack((tmp1,tmp2))

# Adaptive controller with a negative feedback architecture

For this simulation, we want to comprenhensively characterize the proposed control strategy's steady-state behavior for different parameters, such as the actuation gain ($\beta$) and the sequestration rate ($\gamma$). Thus, we will analyze the toggle switch as our study case, which is described by the following ODEs:

\begin{eqnarray}
  \frac{dy_1}{dt} &=& \alpha \frac{K^m}{K^m + y_2^m} - \delta y_1  \\
  \frac{dy_2}{dt} &=& \alpha \frac{K^m}{K^m + y_1^m} - \delta y_2 \\
\end{eqnarray}

If we connect our controller to the $y_1$ gene, the ODEs for the toggle switch will be modified as follows

\begin{eqnarray}
  \frac{dy_1}{dt} &=& \alpha \frac{K^m}{K^m + y_2^m} - \delta y_1 + \beta u_1 \\
  \frac{dy_2}{dt} &=& \alpha \frac{K^m}{K^m + y_1^m} - \delta y_2 \\
\end{eqnarray}

Moreover, the ODEs that describe the dynamics of our controller will be

\begin{eqnarray}
  \frac{du_1}{dt} = kx - \delta u_1 - \gamma u_1 u_2, \space \frac{du_2}{dt} = \xi \frac{K^m}{K^m + y_1^m} - \delta u_2 - \gamma u_1 u_2, \space  \frac{dx}{dt} = \theta \frac{K^m}{K^m + y_1^m} - \delta x
\end{eqnarray}

## Finding the steady state

We can find the nullclines equations from the ODEs that describe the toggle switch:

\begin{eqnarray}
\bar y_2 &=& \frac{\alpha}{\delta} \left (\frac{K^m}{K^m + \bar y_1^m} \right ) \\
\bar y_2 &=& \sqrt[m]{\frac{\alpha K^m}{\delta \bar y_1 - \beta \bar u_1} - K^m}
\end{eqnarray}

Similarly, from the ODEs that describe the adaptive controller, we can find $\bar u_1$ as a function of $\bar y_1$,
\begin{eqnarray}
\bar u_2 &= \frac{f_1 - \delta \bar u_1}{\gamma \bar u_1} = \frac{f_2}{\gamma \bar u_1 + \delta},
\end{eqnarray}
where $f_1 = k \bar x$, $f_2 = \xi \frac{K^m}{K^m + \bar y_1^m}$, and $\bar x = \frac{\theta}{\delta} \frac{K^m}{K^m + \bar y_1^m}$.

This leads to a second order polynomial, $P(\bar u_1) = \bar u_1^2 + A \bar u_1 + B = 0$, and leads to a single positive real solution
\begin{eqnarray}
\bar u_1 &= \frac{-A + \sqrt{A^2 - 4B}}{2}
\end{eqnarray}
where $A = \frac{f_2-f_1}{\delta} + \frac{\delta}{\gamma}$, and $B = - \frac{f_1}{\gamma}$.




In [ ]:
#@title Negative feedback architecture

# Ordinary Differential Equations
def IFFL_NF(X, t, a, K, m, d, b, k, xi, th, g):
    """
    Right hand desribe production and degradation and return dx/dt
    """
    y1, y2, u1, u2, x = X

    return (
      [
          a * K**m / (K**m + y2**m) - d*y1 + b*u1, # d y1 / dt =
          a * K**m / (K**m + y1**m) - d*y2  , # d y2 / dt =
          k * x - d*u1 - g*u1*u2, # d u1 / dt =
          xi * K**m / (K**m + y1**m) - d*u2 - g*u1*u2, # d u2 / dt =
          th * K**m / (K**m + y1**m) - d*x, # d x / dt =
      ]
  )

# Nullclines equations
def Compute_IO(y, a, K, m, d, b, k, xi, th, g):
  x = th/d*K**m/(K**m + y**m)
  f1 = k*x
  f2 = xi*K**m/(K**m + y**m)

  A = (f2-f1)/d + d/g
  B = -f1/g

  u1 = 0.5*(-A + (A**2 - 4*B)**0.5)

  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m) # (1) Eq y1(y2) (hill + b*u1) / d
  y2_2 = a/d*K**m/(K**m + y**m)

  return (y2_1,y2_2, u1)

def Compute_IO_2(y, a, K, m, d, b, k, xi, th, g):
    x = th/d*K**m/(K**m + y**m)
    f1 = k*x
    f2 = xi*K**m/(K**m + y**m)

    A = (f2-f1)/d + d/g
    B = -f1/g

    u1 = 0.5*(-A + (A**2 - 4*B)**0.5)

    ym, tmp = intersection(y, a + b*u1, y, d*y)
    #print(ym)

    y1_1 = np.linspace(0,ym[0]*1.2,len(y))

    y2_2 = a/d*K**m/(K**m + y**m)

    return (y2_2, y1_1)

def Compute_IO_1(y, a, K, m, d, b, k, xi, th, g):
    x = th/d*K**m/(K**m + y**m)
    f1 = k*x
    f2 = xi*K**m/(K**m + y**m)

    A = (f2-f1)/d + d/g
    B = -f1/g

    u1 = 0.5*(-A + (A**2 - 4*B)**0.5)

    y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)
    return (y2_1)

def simple_propensity(propensities, population, t, a, K, m, d, b, k, xi, th, g, Omega):
    y1, y2, u1, u2, x = population

    # Toggle switch's equations
    propensities[0] = a * Omega * (K*Omega)**m/((K*Omega)**m + y2**m) # 0 -> Y1
    propensities[1] = d * y1 # Y1 -> 0
    propensities[2] = a * Omega * (K*Omega)**m/((K*Omega)**m + y1**m) # 0 -> Y1
    propensities[3] = d * y2 # Y2 -> 0
    propensities[4] = b * u1 # 0 -> Y1

    # Adaptive controller's equations
    propensities[5] = k * x # X -> X + U1
    propensities[6] = d * u1 # U1 -> 0
    propensities[7] = xi * Omega * (K*Omega)**m/((K*Omega)**m + y1**m) # Y1 -> Y1 + U2
    propensities[8] = d * u2 # U2 -> 0
    propensities[9] = th * Omega * (K*Omega)**m/((K*Omega)**m + y1**m) # Y1 -> Y1 + X
    propensities[10] = d * x # X -> 0
    propensities[11] = g * u1 * u2 / Omega # U1 + U2 -> 0

# Stoichiometry  matrix of the toggle switch
# Y1, Y2
S1 = np.array(
    [
     [ 1, 0],  # 0 -> Y1
     [-1, 0],  # Y1 -> 0
     [ 0, 1],  # 0 -> Y2
     [ 0,-1],  # Y2 -> 0
     [ 1, 0],  # 0 -> Y1
    ],
    dtype=int,
)
#print(S1)

# Stoichiometry  matrix of the adaptive controller
# U1, U2, X
S2 = np.array(
    [
     [ 1, 0, 0], # X -> U1
     [-1, 0, 0], # U1 -> 0
     [ 0, 1, 0], # Y1 -> U2
     [ 0,-1, 0], # U2 -> 0
     [ 0, 0, 1], # Y1 -> X
     [ 0, 0,-1], # X -> 0
     [-1,-1, 0], # U1 + u2 -> 0
    ],
    dtype=int,
)

# Stoichiometry  matrix of closed-loop system
simple_update = JoinMatrices(S1,S2)

## The adaptive controller enables a biased output

In [ ]:
#@title Dynamics of the closed-loop system
a = 2.2 # production rate [uM/h]
K = 1 # Kappa [uM]
m = 3 # 3 cooperativity
d = 1 # delta [1/h]
k = 1 # production rate [1/h]
g = 100 # sequestration rate [1/h/uM]
th = d # production rate [uM/h]
xi = 1 # production rate [uM/h]
b = 1 # gain [1/h]

# Time interval
t = np.linspace(0,20,400)

# Set up plots
fig_size = [320, 225] # to visualize
plots = []
for i in range(2):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_axis_label = 'time [h]', y_axis_label = 'y [uM]'),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"

# Initial conditions (combinatory)
vecA = np.linspace(1,3,4)
vecB = np.ones(4)*2.9
vecC = np.copy(vecA)
vecC[::1] += 0.1
vec_y1 = np.concatenate([vecB, vecA])
vec_y2 = np.concatenate([vecA, vecC])

# Toggle switch without the controller
args = (a, K, m, d, b*0, k, xi, th, g)

for ii in range(len(vec_y1)):

  x01 = np.array([vec_y1[ii] , vec_y2[ii] , 0., 0., 0.])

  # Solving the ODEs
  x1 = scipy.integrate.odeint(IFFL_NF, x01, t, args=args)
  x1 = x1.transpose()

  # Ploting
  plots[0].line(t,x1[0,:],line_width=3, color="black", legend_label = 'y1')
  plots[0].line(t,x1[1,:],line_width=3, color="orange", legend_label = 'y2')

plots[0].title.text = 'Without controller'

# Toggle switch with controller
args = (a, K, m, d, b*3, k, xi, th, g)
for ii in range(len(vec_y1)):

  x01 = np.array([vec_y1[ii] , vec_y2[ii] , 0., 0., 0.])

  # Solving the ODEs
  x1 = scipy.integrate.odeint(IFFL_NF, x01, t, args=args)
  x1 = x1.transpose()

  # Ploting
  plots[1].line(t,x1[0,:],line_width=3, color="black", legend_label = 'y1')
  plots[1].line(t,x1[1,:],line_width=3, color="orange", legend_label = 'y2')

plots[1].title.text = 'With IFFL controller'

# Tidy up and set export settings
for i in range(2):
  plots[i].legend.background_fill_alpha = 0.00
  plots[i].legend.location = 'bottom_right'
  #plots[i].output_backend = 'svg'

bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
#@title Stochastic simulation
a = 2.2 # production rate [uM/h]
K = 1 # Kappa [uM]
m = 3 # 3 cooperativity
d = 1 # delta [1/h]
k = 1 # production rate [1/h]
g = 10 # sequestration rate [1/h/uM]
th = d # production rate [uM/h]
xi = 1 # production rate [uM/h]
b = 1 # gain [1/h]
Omega = 100 # 600

time_points = np.linspace(0, 20, 101)
population_0 = np.zeros(5, dtype=int)

# Parameters to analyze
args1 = (a, K, m, d, b*0, k, xi, th, g, Omega)
args2 = (a, K, m, d, b*3, k, xi, th, g, Omega)

# Solving with Gillespie algorithm
samples1 = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=250, args=args1, n_threads=4,progress_bar=False)
samples2 = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=250, args=args2, n_threads=4,progress_bar=False)

# Set up plots
fig_size = [320, 225] # to visualize
plots = []
for i in range(4):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          x_axis_label = 'time [h]'),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"

# Plotting the results of the stochastic simulation (10 trajectories)
for i in range(100):
    plots[0].line(time_points,samples1[i,:,0],line_width=1, color="black", alpha= 0.2, line_join="bevel") #
    plots[1].line(time_points,samples1[i,:,1],line_width=1, color="black", alpha= 0.2, line_join="bevel") #
    plots[2].line(time_points,samples2[i,:,0],line_width=1, color="black", alpha= 0.2, line_join="bevel") #
    plots[3].line(time_points,samples2[i,:,1],line_width=1, color="black", alpha= 0.2, line_join="bevel") #

# Filter depending on the steady-state value
indices_samples = [(0, samples1[:10, :, 0]), (1, samples1[:10, :, 1]),
                   (2, samples2[:10, :, 0]), (3, samples2[:10, :, 1])]

# Iterate over samples and plot the mean
for i, sample in indices_samples:
    copy1 = sample[sample[:, -1] > 100]
    copy2 = sample[sample[:, -1] < 100]

    # Plotting the mean for samples above and below the threshold (100)
    plots[i].line(time_points, copy1.mean(axis=0), line_width=3, color='orange', line_join='bevel')
    plots[i].line(time_points, copy2.mean(axis=0), line_width=3, color='orange', line_join='bevel')

# Tidy up and set export settings
plots[0].title.text = 'Without controller'
plots[2].title.text = 'With controller'
plots[0].yaxis.axis_label = 'y1 [# molecules]'
plots[1].yaxis.axis_label = 'y2 [# molecules]'
plots[2].yaxis.axis_label = 'y1 [# molecules]'
plots[3].yaxis.axis_label = 'y2 [# molecules]'
# for i in range(4):
#   plots[i].output_backend = "svg"

bokeh.io.show(bokeh.layouts.gridplot([
    [plots[0], plots[1]],
    [plots[2], plots[3]]
]))

<ipython-input-8-aab81bb5e626>:52: RuntimeWarning: Mean of empty slice.
  plots[i].line(time_points, copy2.mean(axis=0), line_width=3, color='orange', line_join='bevel')
<ipython-input-8-aab81bb5e626>:51: RuntimeWarning: Mean of empty slice.
  plots[i].line(time_points, copy1.mean(axis=0), line_width=3, color='orange', line_join='bevel')


In [ ]:
#@title The adaptive controller generates a skewed probability distribution that favors the production of $Y_1$ species

# IMPORTANT: run the previous code chunk ("Stochastic simulation") before running the present chunk

# Set up plots
fig_size = [320, 225] # to visualize
x_range = (-300, 300)
y_range = (0, 0.025)
plots = []
for i in range(3):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                                       x_range=x_range, y_range=y_range,
                                       x_axis_label='y1 - y2 [# molecules]', y_axis_label='p'))
    plots[i].axis.major_label_text_font_size = "10pt"

# Bin size for histogram
bin0 = np.arange(-3 * Omega, 3 * Omega + 6, 8) #8

# Some extra details for the plot
samples = [samples1, samples2]
colors = ["grey", "orange"]
titles = ['without controller', 'with controller']
legends = ['β = 0', 'β = 3']

# Computing the histograms
for i, (sample, color, title) in enumerate(zip(samples, colors, titles)):
    hist, bin_edges = np.histogram(sample[:, -1, 0] - sample[:, -1, 1], bin0, density=True)

    plots[i].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color=color, line_color=color, fill_alpha=0.2, line_alpha=0.)
    plots[i].step(bin_edges[:-1], hist, line_width=1, mode="after", color=color)
    plots[i].title.text = title

    plots[2].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color=color, line_color=color, fill_alpha=0.2, line_alpha=0.)
    plots[2].step(bin_edges[:-1], hist, line_width=1, mode="after", color=color, legend_label=legends[i])

# Tidy up and set export settings
plots[2].title.text = 'Visualizing the biased cell fate'
plots[2].legend.location = 'top'
plots[2].legend.background_fill_alpha = 0.0
# for i in range(3):
#   plots[i].output_backend = 'svg'

bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
#@title Adaptive controller in action: phase plane visualization

# IMPORTANT: run the previous code chunk ("Stochastic simulation") before running the present chunk

# Set up plots
fig_size = [320, 225] # to visualize
x_range = Range1d(0, 3.1)
y_range = Range1d(0, 3.1)
plots = []
for i in range(2):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range = x_range, y_range = y_range,
                          x_axis_label = 'y1 [uM]', y_axis_label = 'y2 [uM]'
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"

# Computing the nullclines
y = np.linspace(0,3,201)
y2_1, y2_2, u1 = Compute_IO(y, a, K, m, d, b*0, k, xi, th, g)

plots[0].line(y,y2_1,line_width=2, color="black", ) #
plots[0].line(y,y2_2,line_width=2, color="black", ) #

y1_i, y2_i = intersection(y, y2_1, y, y2_2)
plots[0].scatter(y1_i, y2_i, size = 7, color = 'black')

y2_1, y2_2, u1 = Compute_IO(y, a, K, m, d, b*3, k, xi, th, g)
plots[1].line(y,y2_1,line_width=2, color="black", ) #
plots[1].line(y,y2_2,line_width=2, color="black", ) #

# Assigning "red" if the steady-state value is near the y2 equilibrium
# Assigning "blue" if the steady-state value is near the y1 equilibrium
for i in range(8):
    sol = samples1[i,:,:]/Omega
    ratio = sol[-1,0]/sol[-1,1]
    if ratio > 1:
        plots[0].line(sol[:,0],sol[:,1],line_width=3, color="blue",alpha= 0.1, line_join="bevel") #
    elif ratio < 1:
        plots[0].line(sol[:,0],sol[:,1],line_width=3, color="red",alpha= 0.1, line_join="bevel") #
    else:
        plots[0].line(sol[:,0],sol[:,1],line_width=3, color="black",alpha= 0.1, line_join="bevel") #

    sol = samples2[i,:,:]/Omega
    ratio = sol[-1,0]/sol[-1,1]
    if ratio > 1:
        plots[1].line(sol[:,0],sol[:,1],line_width=3, color="blue",alpha= 0.1, line_join="bevel") #
    elif ratio < 1:
        plots[1].line(sol[:,0],sol[:,1],line_width=3, color="red",alpha= 0.1, line_join="bevel") #
    else:
        plots[1].line(sol[:,0],sol[:,1],line_width=3, color="black",alpha= 0.1, line_join="bevel") #

# Tidy up and set export settings
plots[0].title.text = 'without controller'
plots[1].title.text = 'with IFFL controller'
for i in range(2):
  plots[i].output_backend = 'svg'

bokeh.io.show(bokeh.layouts.row(plots))

<ipython-input-3-887fe4e8fbaa>:31: RuntimeWarning: divide by zero encountered in divide
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m) # (1) Eq y1(y2) (hill + b*u1) / d
<ipython-input-3-887fe4e8fbaa>:31: RuntimeWarning: invalid value encountered in power
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m) # (1) Eq y1(y2) (hill + b*u1) / d


In [ ]:
#@title Higher values of gain (β) generate higher probabilities in the desired cell fate
a = 2.2 # production rate [uM/h]
K = 1 # Kappa [uM]
m = 3 # 3 cooperativity
d = 1 # delta [1/h]
k = 1 # production rate [1/h]
g = 100 # sequestration rate
th = d # production rate [uM/h]
xi = 1 # production rate [uM/h]
b = 1 # gain [1/h]
Omega = 100 # 600

time_points = np.linspace(0, 20, 101)
population_0 = np.zeros(5, dtype=int)

# Set up plots
fig_size = [320, 225] # to visualize
x_range = (-300, 300)
y_range = (0, 0.03)

plots = []
for i in range(4):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range,
                          x_axis_label = 'y1 - y2 [# molecules]', y_axis_label = 'p'
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

# Parameters to analyze
b_values = [0, 1, 2.5, 5]
args = [(a, K, m, d, b * value, k, xi, th, g, Omega) for value in b_values]

# Solving with Gillespie algorithm for each set of parameters
samples = [biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0, time_points, size=250, args=arg, n_threads=4, progress_bar=False) for arg in args]

# Bin size definition
bin0 = np.arange(-3 * Omega, 3 * Omega + 6, 8) # 8

# Computing the histograms and plotting
for i, sample in enumerate(samples):
    hist, bin_edges = np.histogram(sample[:, -1, 0] - sample[:, -1, 1], bin0, density=True)
    # Color coding
    if (i == 0): color = 'grey'
    else: color = 'orange'
    plots[i].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color=color, line_color=color, fill_alpha=0.2, line_alpha=0.)
    plots[i].step(bin_edges[:-1], hist, line_width=1, mode="after", color=color)
    plots[i].title.text = f'β = {b_values[i]}'

# Tidy up and set export settings
#plots[0].legend.location = 'top'
#plots[0].legend.background_fill_alpha = 0.0
# for i in range(4):
#   plots[i].output_backend = 'svg'

bokeh.io.show(bokeh.layouts.row(plots))

## Steady state analysis

In [ ]:
#@title Preliminary steady state analysis
a = 2.2 # production rate [uM/h]
K = 1 # Kappa [uM]
m = 3 # 3 cooperativity
d = 1 # delta [1/h]
k = 1 # production rate [1/h]
g = 100 # sequestration rate [1/h/uM]
th = d # production rate [uM/h]
xi = 1 # production rate [uM/h]
vec_b = [1,5] # gain [1/h]
Omega = 100 # 600

# Time interval
t = np.linspace(0,20,400)

# Set up plots
fig_size = [320, 225] # to visualize
x_range = (0, 3)
y_range = (0, 3)

plots = []
for i in range(2):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,y_range=y_range,
                          x_axis_label = 'y1 [uM]', y_axis_label = 'y2 [uM]'
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"

# Plotting different gain values to explore deviations to the equilibrium landscape
for i, b in enumerate(vec_b):
    y1_2 = np.logspace(-5, 1, 500)

    # Plot without the adaptive controller
    y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, b * 0, k, xi, th, g)
    y2_1 = Compute_IO_1(y1_1, a, K, m, d, b * 0, k, xi, th, g)
    y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

    plots[i].line(y1_1, y2_1, line_width=2, color="black", line_dash="dashed", legend_label='w/o IFFL')
    plots[i].line(y1_2, y2_2, line_width=2, color="black", line_dash="dashed")
    plots[i].scatter(y1_i, y2_i, size=6, color='red', fill_color='white', line_width=2)

    # Plot with the adaptive controller
    y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, b, k, xi, th, g)
    y2_1 = Compute_IO_1(y1_1, a, K, m, d, b, k, xi, th, g)
    y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

    plots[i].line(y1_1, y2_1, line_width=2, color="black", legend_label='w IFFL')
    plots[i].line(y1_2, y2_2, line_width=2, color="black")
    plots[i].scatter(y1_i, y2_i, size=6, color='blue' if b == 1 else 'red', fill_color='white', line_width=2)

    # Titles and labels
    plots[i].title.text = f'β = {b if b != 0 else 1}'
    plots[i].legend.background_fill_alpha = 0.0

# Tidy up and set export settings
# for i in range(2):
#   plots[i].output_backend = 'svg'

bokeh.io.show(bokeh.layouts.row(plots))

<ipython-input-3-887fe4e8fbaa>:65: RuntimeWarning: divide by zero encountered in divide
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)
<ipython-input-3-887fe4e8fbaa>:65: RuntimeWarning: invalid value encountered in power
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)


In [ ]:
#@title How does the controller alters the equilibrium landscape?
a = 2.2 # production rate [uM/h]
K = 1 # Kappa [uM]
m = 3 # 3 cooperativity
d = 1 # delta [1/h]
k = 1 # production rate [1/h]
gv = (10,100,1000) # sequestration rate [1/h/uM]
th = d # production rate [uM/h]
xi = 1 # production rate [uM/h]
b = 1 # gain [1/h]

# Defining a color pallete
palette = bokeh.palettes.grey(5)[::-1]

# Set up plots
fig_size = [320, 225] # to visualize
x_range = (0, 5.1)
y_range = (2.1, 2.51)

plots = []
for i in range(2):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_axis_label = 'β',
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"

# Comprenhensive characterization on how the gain affects the equilibrium landscape
vec = np.linspace(0,5,7)
y1_2 = np.linspace(0,3,300)

for kk, gi in enumerate(gv):
    out1 = np.empty((3,len(vec),))*np.nan
    out2 = np.empty((3,len(vec),))*np.nan
    for i, bi in enumerate(vec):
        y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, bi, k, xi, th, gi)
        y2_1 = Compute_IO_1(y1_1, a, K, m, d, bi, k, xi, th, gi)
        y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

        if len(y1_i)>0:
            out1[0,i] = np.max(y1_i)
            out2[0,i] = np.max(y2_i)
            if len(y1_i)==1:
                out1[2,i] = np.max(y1_i)
                out2[2,i] = np.max(y2_i)
            else:
                out1[1,i] = np.max(y1_i)
                out2[1,i] = np.max(y2_i)

    plots[0].line(vec,out1[0,:],line_width=2, color=palette[kk+1],) #
    plots[0].scatter(vec,out1[1,:],line_width=1, size = 7, color=palette[kk+1], legend_label = f'{gi/g} γ ') #
    plots[0].scatter(vec,out1[2,:],line_width=1, size = 7, color=palette[kk+1],fill_color = 'white') #

    plots[1].line(vec,out2[0,:],line_width=2, color=palette[kk+1], ) #
    plots[1].scatter(vec,out2[1,:],line_width=1, size = 7, color=palette[kk+1], ) #
    plots[1].scatter(vec,out2[2,:],line_width=1, size = 7, color=palette[kk+1],fill_color = 'white') #

# Reference plot
y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, b*0, k, xi, th, gi)
y2_1 = Compute_IO_1(y1_1, a, K, m, d, b*0, k, xi, th, gi)
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)
hline = bokeh.models.Span(location=np.max(y1_i), dimension='width', line_color='grey', line_width=1, line_dash = 'dashed')
plots[0].renderers.extend([hline])
hline = bokeh.models.Span(location=np.max(y2_i), dimension='width', line_color='grey', line_width=1, line_dash = 'dashed')
plots[1].renderers.extend([hline])

# Tidy up and set export settings
plots[0].y_range = Range1d(2.1, 2.51)
plots[1].y_range = Range1d(0, 2.5)

plots[0].legend.location = 'top_left'
plots[0].legend.background_fill_alpha = 0.0

plots[0].yaxis.axis_label = 'y1'
plots[1].yaxis.axis_label = 'y2'

# for i in range(2):
#   plots[i].output_backend = 'svg'

bokeh.io.show(bokeh.layouts.row(plots))

<ipython-input-52-60763a872651>:69: RuntimeWarning: divide by zero encountered in divide
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)
<ipython-input-52-60763a872651>:69: RuntimeWarning: invalid value encountered in power
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)


In [ ]:
#@title Tradef-off between the sequestration rate ($\gamma$) and control gain ($\beta$)
a = 2.2  # production rate [uM/h]
K = 1    # Kappa [uM]
m = 3    # cooperativity
d = 1    # delta [1/h]
k = 1    # production rate [1/h]
g = 100  # sequestration rate [1/h/uM]
th = d   # production rate [uM/h]
xi = 1   # production rate [uM/h]
vec_b = [0, 2, 2]  # gain values
Omega = 100  # 600

time_points = np.linspace(0, 20, 101)
population_0 = np.zeros(5, dtype=int)

# Set up plots
fig_size = [300, 225]

# Set up plots for probability analysis
row0 = []
for i in range(3):
    row0.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                                       x_range=Range1d(-300, 300), y_range=Range1d(0, 0.02)))
    row0[i].axis.major_label_text_font_size = "10pt"

# Set up plots for nullclines analysis
row1 = []
for i in range(3):
    row1.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                                       x_range=Range1d(0, 3.1), y_range=Range1d(0, 3.1)))
    row1[i].axis.major_label_text_font_size = "10pt"

# Parameters to analyze
args_list = [
    (a, K, m, d, vec_b[0], k, xi, th, g, Omega),
    (a, K, m, d, vec_b[1], k, xi, th, g, Omega),
    (a, K, m, d, vec_b[2], k, xi, th, g*10, Omega)
]

# Solving with Gillespie algorithm
samples = [biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0, time_points, size=250, args=args, n_threads=4, progress_bar=False) for args in args_list]

# Plotting histograms
bin0 = np.arange(-3*Omega, 3*Omega+6, 8)
for i, b in enumerate(vec_b):
    hist, bin_edges = np.histogram(samples[i][:, -1, 0] - samples[i][:, -1, 1], bin0, density=True)
    row0[i].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color=["grey", "orange", "purple"][i], line_color=["grey", "orange", "purple"][i], fill_alpha=0.2, line_alpha=0.)
    row0[i].step(bin_edges[:-1], hist, line_width=1, mode="after", color=["grey", "orange", "purple"][i])
    row0[i].title.text = f'β = {b}' if b != vec_b[-1] else f'β = {b}, γ = 1000'
    row0[i].xaxis.axis_label = 'y1 - y2 [# molecules]'
    row0[i].yaxis.axis_label = 'p'

# Plotting nullclines
for i, b in enumerate(vec_b):
    y1_2 = np.linspace(0, 3, 300)
    y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, b, k, xi, th, g)
    y2_1 = Compute_IO_1(y1_1, a, K, m, d, b, k, xi, th, g)
    y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

    row1[i].line(y1_1, y2_1, line_width=2, color='black', alpha=0.2)
    row1[i].line(y1_2, y2_2, line_width=2, color='black', alpha=0.2)
    row1[i].scatter(y1_i, y2_i, line_width=1, color=["grey", "orange", "purple"][i], size=7)
    row1[i].xaxis.axis_label = 'y1 [uM]'
    row1[i].yaxis.axis_label = 'y2 [uM]'

# Visualization
bokeh.io.show(bokeh.layouts.gridplot([[row0[0], row0[1], row0[2]], [row1[0], row1[1], row1[2]]]))


<ipython-input-3-887fe4e8fbaa>:65: RuntimeWarning: divide by zero encountered in divide
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)
<ipython-input-3-887fe4e8fbaa>:65: RuntimeWarning: invalid value encountered in power
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)


## Optimal parameter characterization

In the fast sequestration regime ($\gamma \to \infty$), we can approximate the steady state as
\begin{eqnarray}
u_1 &= \max \left ( 0, \frac{1}{\delta} \left ( \frac{k \theta}{\delta} - \xi\right ) \frac{K^m}{K^m + \bar y_1^m} \right ) = \max \left (0, \frac{k\theta}{\delta^2} \left ( 1 - \frac{\delta \xi}{k \theta} \right ) \frac{K^m}{K^m + \bar y_1^m} \right )
\end{eqnarray}


Similarly, we can approximate the dynamics of the controller in the fast sequestration regime ($\gamma \to \infty$), and find that
\begin{eqnarray}
u_1(t) & = \max \left ( 0, L^{-1} \left ( \frac{\theta k}{(s + \delta)^2}  \left (1 - \frac{\delta \xi}{k \theta} - \frac{ \xi}{k \theta} s \right) Y^*_1(s)\right )\right )
\end{eqnarray}

If we set $\xi \cdot \delta = k \cdot \theta$, we notice that the system asymptotically resembles a low-pass filter and a temporal derivative. Therefore, we define a parameter $r$, such that $r = \xi \delta / k \theta$, and define an ideal adaptive metric as $r = 1$.

In [ ]:
#@title Can we deviate from the adaptive metric?
a = 2.2  # production rate [uM/h]
K = 1    # Kappa [uM]
m = 3    # cooperativity
d = 1    # delta [1/h]
k = 1    # production rate [1/h]
g = 100  # sequestration rate [1/h/uM]
th = d   # production rate [uM/h]
xi = 1   # production rate [uM/h]
b = 2    # gain value [1/h]
Omega = 100

time_points = np.linspace(0, 20, 101)
population_0 = np.zeros(5, dtype=int)

# Parameters to analyze
args_list = [
    (a, K, m, d, b * 0, k, xi, th, g, Omega),
    (a, K, m, d, b, k, xi, th, g, Omega),
    (a, K, m, d, b, k, xi * 2, th, g, Omega)
]

samples_list = [
    biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0, time_points, size=1000, args=args, n_threads=5, progress_bar=False)
    for args in args_list
]

# Set up plots for probability analysis
fig_size = [300, 225]  # to visualize

row0 = []
for i in range(3):
    row0.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                                      x_range=Range1d(-300, 300), y_range=Range1d(0, 0.02),
                                      x_axis_label = 'y1 - y2 [# molecules]', y_axis_label = 'p'))
    row0[i].axis.major_label_text_font_size = "10pt"

# Set up plots for nullcline analysis
row1 = []
for i in range(3):
    row1.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                                      x_range=Range1d(0, 3.1), y_range=Range1d(0, 3.1),
                                      x_axis_label = 'y1 [uM]', y_axis_label = 'y2 [uM]'))
    row1[i].axis.major_label_text_font_size = "10pt"

# Extra details for the plots
colors = ["grey", "orange", "purple"]
tags = ['Without controller', f'r = {(xi*2)*d/(th*k)}', f'r = {xi*d/(th*k)}']
x_labels = ['y2 - y1 [# molecules]', 'y2 - y1 [# molecules]', 'y2 - y1 [# molecules]']

# Bin size definition
bin0 = np.arange(-3 * Omega, 3 * Omega + 6, 8)

for i, (samples, color, title) in enumerate(zip(samples_list, colors, tags)):
    hist, bin_edges = np.histogram(samples[:, -1, 0] - samples[:, -1, 1], bin0, density=True)
    row0[i].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color=color, line_color=color, fill_alpha=0.2, line_alpha=0)
    row0[i].step(bin_edges[:-1], hist, line_width=1, mode="after", color=color)
    row0[i].title.text = title

    y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, b if i > 0 else 0, k, xi if i != 2 else xi*2, th, g)
    y2_1 = Compute_IO_1(y1_1, a, K, m, d, b if i > 0 else 0, k, xi if i != 2 else xi*2, th, g)
    y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

    row1[i].line(y1_1, y2_1, line_width=2, color='black', alpha=0.2)
    row1[i].line(y1_2, y2_2, line_width=2, color='black', alpha=0.2)
    row1[i].scatter(y1_i, y2_i, line_width=1, color=color, size=7)

bokeh.io.show(bokeh.layouts.gridplot([[row0[0], row0[1], row0[2]], [row1[0], row1[1], row1[2]]]))

<ipython-input-4-b571fed29f76>:69: RuntimeWarning: divide by zero encountered in divide
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)
<ipython-input-4-b571fed29f76>:69: RuntimeWarning: invalid value encountered in power
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)


In [ ]:
#@title Quantification of the deviation's effect on the equilibrium
a = 2.2 # production rate [uM/h]
K = 1 # Kappa [uM]
m = 3 # 3 cooperativity
d = 1 # delta [1/h]
k = 1 # production rate [1/h]
gv = (10,100,1000) # sequestration rate [1/h/uM]
th = d # production rate [uM/h]
xi = 1 # production rate [uM/h]
b = 2 # gain [1/h]

palette = bokeh.palettes.grey(5)[::-1]

# Varying r metric through changes in the "k" production rate
k_vec = np.linspace(0.5,2,7)
vec = d*xi/th/k_vec

# Varying the state variable
y1_2 = np.linspace(0,3,300)

# Set up plots
fig_size = [300, 225] # to visualize
x_range = (0.4, 2.1)
y_range = (2.1, 2.51)

plots = []
for i in range(2):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,y_range=y_range,
                          x_axis_label = 'r'
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"

# Characterizing the deviation to the equilibrium value with respect to the "r" adaptive metric
for kk, gi in enumerate(gv):
    out1 = np.empty((3,len(vec),))*np.nan
    out2 = np.empty((3,len(vec),))*np.nan
    for i, ki in enumerate(k_vec):
        y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, b, ki, xi, th, gi)
        y2_1 = Compute_IO_1(y1_1, a, K, m, d, b, ki, xi, th, gi)
        y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

        if len(y1_i)>0:
            out1[0,i] = np.max(y1_i)
            out2[0,i] = np.max(y2_i)
            if len(y1_i)==1:
                out1[2,i] = np.max(y1_i)
                out2[2,i] = np.max(y2_i)
            else:
                out1[1,i] = np.max(y1_i)
                out2[1,i] = np.max(y2_i)

    plots[0].line(vec,out1[0,:],line_width=2, color=palette[kk+1],) #
    plots[0].scatter(vec,out1[1,:],line_width=1, size = 7, color=palette[kk+1], legend_label = f'{gi/g} γ ') #
    plots[0].scatter(vec,out1[2,:],line_width=1, size = 7, color=palette[kk+1],fill_color = 'white') #

    plots[1].line(vec,out2[0,:],line_width=2, color=palette[kk+1],) #
    plots[1].scatter(vec,out2[1,:],line_width=1, size = 7, color=palette[kk+1],) #
    plots[1].scatter(vec,out2[2,:],line_width=1, size = 7, color=palette[kk+1],fill_color = 'white') #

# Reference plot
y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, b*0, ki, xi, th, gi)
y2_1 = Compute_IO_1(y1_1, a, K, m, d, b*0, ki, xi, th, gi)
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)
hline = bokeh.models.Span(location=np.max(y1_i), dimension='width', line_color='grey', line_width=1, line_dash = 'dashed')
plots[0].renderers.extend([hline])
hline = bokeh.models.Span(location=np.max(y2_i), dimension='width', line_color='grey', line_width=1, line_dash = 'dashed')
plots[1].renderers.extend([hline])

# Tidy up and set export settings
plots[0].y_range = Range1d(2.12, 2.42)
plots[1].y_range = Range1d(0, 2.5)
plots[0].yaxis.axis_label  = 'y1'
plots[1].yaxis.axis_label  = 'y2'
plots[0].legend.background_fill_alpha = 0.0

# for i in range(2):
#   plots[i].output_backend = 'svg'

bokeh.io.show(bokeh.layouts.row(plots))

<ipython-input-4-b571fed29f76>:69: RuntimeWarning: invalid value encountered in power
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)
<ipython-input-4-b571fed29f76>:69: RuntimeWarning: divide by zero encountered in divide
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)


In [ ]:
#@title The adaptive metric ($r = \xi \delta / k \theta$) allows a $\pm$ 10% error margin
a = 2.2  # production rate [uM/h]
K = 1  # Kappa [uM]
m = 3  # cooperativity
d = 1  # delta [1/h]
k = 1  # production rate [1/h]
g = 100  # sequestration rate [1/h/uM]
th = d  # production rate [uM/h]
xi = 1  # production rate [uM/h]
b = 2  # gain [1/h]
Omega = 100  # 600

# Bin size definition
bin0 = np.arange(-3*Omega, 3*Omega+6, 8)

time_points = np.linspace(0, 20, 101)
population_0 = np.zeros(5, dtype=int)

# Parameter variations to analyze
param_variations = {
    '0%': {'xi': 1, 'b': b*0, 'color': 'grey', 'alpha': 0.2, 'label': 'control'},
    '+1%': {'xi': 1.01, 'b': b, 'color': 'orange', 'alpha': 0.2},
    '-1%': {'xi': 0.99, 'b': b, 'color': 'purple', 'alpha': 0.2},
    '+10%': {'xi': 1.1, 'b': b, 'color': 'blue', 'alpha': 0.2},
    '-10%': {'xi': 0.9, 'b': b, 'color': 'green', 'alpha': 0.2},
}

# Computing the histograms
samples = {}
histograms = {}
for label, params in param_variations.items():
    args = (a, K, m, d, params['b'], k, params['xi'], th, g, Omega)
    samples[label] = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0, time_points, size=1000, args=args, n_threads=5, progress_bar=False)
    histograms[label], bin_edges = np.histogram(samples[label][:, -1, 0] - samples[label][:, -1, 1], bin0, density=True)

# Set up plots
fig_size = [300, 225]
x_range = (-300, 300)
y_range = (0, 0.02)

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                                       x_range=x_range, y_range=y_range))
    plots[i].axis.major_label_text_font_size = "10pt"

# Reference nullclines plot
y1_2 = np.logspace(-5, 1, 500)
y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, 0, k, xi, th, g)
y2_1 = Compute_IO_1(y1_1, a, K, m, d, 0, k, xi, th, g)
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

plots[5].line(y1_1, y2_1, line_width=2, color='black', alpha=0.2)
plots[5].line(y1_2, y2_2, line_width=2, color='black', alpha=0.2)
plots[5].scatter(y1_i, y2_i, line_width=1, color='gray', size=7, legend_label='control')

# Plot histograms and nullclines for each condition
for i, (label, params) in enumerate(param_variations.items()):
    plots[i].quad(top=histograms[label], bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color=params['color'], line_color=params['color'], fill_alpha=params['alpha'], line_alpha=0)
    plots[i].step(bin_edges[:-1], histograms[label], line_width=1, mode="after", color=params['color'])
    plots[i].title.text = f'r = {params["xi"]*d/(k*th):.2f}'
    plots[i].xaxis.axis_label = 'y1 - y2 [# molecules]'
    plots[i].yaxis.axis_label = 'p'

    y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, b, k, params['xi'], th, g)
    y2_1 = Compute_IO_1(y1_1, a, K, m, d, b, k, params['xi'], th, g)
    y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)
    plots[5].scatter(y1_i, y2_i, line_width=1, color=params['color'], size=7, legend_label=label)

# Tidy up and set export settings
plots[0].title.text = 'Without controller'
plots[5].xaxis.axis_label = 'y1 [uM]'
plots[5].yaxis.axis_label = 'y2 [uM]'
plots[5].x_range = bokeh.models.Range1d(0, 3.1)
plots[5].y_range = bokeh.models.Range1d(0, 3.1)
plots[5].legend.background_fill_alpha = 0.0

# for i in range(6):
#   plots[i].output_backend = 'svg'

bokeh.io.show(bokeh.layouts.gridplot([[plots[0], plots[1], plots[2]], [plots[3], plots[4], plots[5]]]))

<ipython-input-3-887fe4e8fbaa>:65: RuntimeWarning: divide by zero encountered in divide
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)
<ipython-input-3-887fe4e8fbaa>:65: RuntimeWarning: invalid value encountered in power
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)
